In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
uploaded = files.upload()

In [0]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True



In [3]:
!pip install transformers

     |████████████████████████████████| 450kB 4.7MB/s 
     |████████████████████████████████| 1.0MB 54.9MB/s 
     |████████████████████████████████| 870kB 59.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=d579185fa828ad5776e5612222c386f9664f8dd0fbd8d2e922c0fe7b36a683de
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


In [5]:
len(tokenizer.vocab)
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU.ham to bewakuf hain?')

print(tokens)


# We can numericalize tokens using our vocabulary using `tokenizer.convert_tokens_to_ids`.

# In[5]:


indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)


['Hello', 'WORLD', 'how', 'AR', '##E', 'yo', '##U', '.', 'ham', 'to', 'be', '##wak', '##uf', 'hain', '?']
[31178, 67376, 14796, 50884, 11259, 13672, 12022, 119, 15128, 10114, 10347, 57482, 21598, 106629, 136]


In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)


# We can get the indexes of the special tokens by converting them using the vocabulary...

# In[7]:


init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)


# ...or by explicitly getting them from the tokenizer.

# In[8]:


init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)


# Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

# In[9]:


max_input_length = tokenizer.max_model_input_sizes['bert-base-multilingual-cased']

print(max_input_length)


# Previously we have used the `spaCy` tokenizer to tokenize our examples. However we now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

# In[10]:


def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens


# Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.
# 
# We define the label field as before.

# In[87]:


from torchtext import data


[CLS] [SEP] [PAD] [UNK]
101 102 0 100
101 102 0 100
512


In [7]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField()


# We load the data and create the validation splits as before.

# In[89]:


from torchtext import datasets
fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}
train_data, test_data = data.TabularDataset.splits(
                                        path = './',
                                        train = 'demo_data',
                                        test = 'test_demo_data',
                                        format = 'tsv',
                                        fields = fields,
                                        skip_header = False
)
# train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
# valid_data = train_data
train_data, valid_data = train_data.split(random_state = random.seed(SEED))


# In[90]:


print(vars(train_data[1]))


{'text': [33478, 10112, 10126, 34709, 37821, 25085, 10751, 39486, 10108, 20442, 12103], 'label': '1'}


In [8]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")


# We can check an example and ensure that the text has already been numericalized.

# In[92]:


print(vars(train_data.examples[6]))


# We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

# In[93]:


tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)


# Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

# In[94]:


LABEL.build_vocab(train_data)


# In[95]:


print(LABEL.vocab.stoi)


# As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

# In[96]:


BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Number of training examples: 9799
Number of validation examples: 4200
Number of testing examples: 1132
{'text': [186, 10123, 15127, 18926, 10124, 10472, 169, 53069, 37949, 77236, 10345, 10143, 10206, 119, 71959, 18926, 37949, 77236, 10116, 62137, 106629, 36769, 70273, 18926, 82612, 10113, 13173, 10263, 10114, 12361, 10418, 11643, 19626, 10550, 15838, 10284], 'label': '1'}
['r', '##t', 'my', 'vote', 'is', 'not', 'a', 'charity', 'khai', '##raat', 'or', 'da', '##an', '.', 'mera', 'vote', 'khai', '##raat', '##i', 'nahi', 'hain', 'agar', 'mein', 'vote', 'kart', '##a', 'ho', '##on', 'to', 'mu', '##j', '##he', 'us', '##ke', 'bad', '##le']
defaultdict(<function _default_unk_index at 0x7f5eb3923158>, {'1': 0, '2': 1, '0': 2})


In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key=lambda x: len(x.text), 
    batch_size = BATCH_SIZE, 
    device = device)


In [0]:

from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-multilingual-cased')


In [0]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output


In [12]:
HIDDEN_DIM = 256
OUTPUT_DIM = 3
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)


# We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

# In[100]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


# In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model and if they're a part of the `bert` transformer model, we set `requires_grad = False`. 

# In[101]:


for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False


# We can now see that our model has under 3M trainable parameters, making it almost comparable to the `FastText` model. However, the text still has to propagate through the transformer which causes training to take considerably longer.

# In[102]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 181,796,355 trainable parameters
The model has 3,942,915 trainable parameters


In [13]:

for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
rnn.weight_ih_l2
rnn.weight_hh_l2
rnn.bias_ih_l2
rnn.bias_hh_l2
rnn.weight_ih_l2_reverse
rnn.weight_hh_l2_reverse
rnn.bias_ih_l2_reverse
rnn.bias_hh_l2_reverse
out.weight
out.bias


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())


# In[105]:


criterion = nn.CrossEntropyLoss()


# Place the model and criterion onto the GPU (if available)

# In[106]:


model = model.to(device)
criterion = criterion.to(device)


# Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

# In[107]:


def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])


# In[108]:


def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


# In[110]:


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [16]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



Epoch: 01 | Epoch Time: 0m 56s
	Train Loss: 1.014 | Train Acc: 46.45%
	 Val. Loss: 0.969 |  Val. Acc: 50.13%
Epoch: 02 | Epoch Time: 1m 1s
	Train Loss: 0.945 | Train Acc: 52.28%
	 Val. Loss: 0.950 |  Val. Acc: 51.69%
Epoch: 03 | Epoch Time: 1m 4s
	Train Loss: 0.916 | Train Acc: 54.57%
	 Val. Loss: 0.940 |  Val. Acc: 54.42%
Epoch: 04 | Epoch Time: 1m 3s
	Train Loss: 0.901 | Train Acc: 55.84%
	 Val. Loss: 0.925 |  Val. Acc: 54.94%
Epoch: 05 | Epoch Time: 1m 3s
	Train Loss: 0.866 | Train Acc: 58.85%
	 Val. Loss: 0.896 |  Val. Acc: 57.22%
Epoch: 06 | Epoch Time: 1m 4s
	Train Loss: 0.854 | Train Acc: 59.97%
	 Val. Loss: 0.897 |  Val. Acc: 56.67%
Epoch: 07 | Epoch Time: 1m 3s
	Train Loss: 0.832 | Train Acc: 61.38%
	 Val. Loss: 0.934 |  Val. Acc: 54.02%
Epoch: 08 | Epoch Time: 1m 3s
	Train Loss: 0.816 | Train Acc: 62.06%
	 Val. Loss: 0.885 |  Val. Acc: 57.71%
Epoch: 09 | Epoch Time: 1m 5s
	Train Loss: 0.786 | Train Acc: 64.06%
	 Val. Loss: 0.903 |  Val. Acc: 58.30%
Epoch: 10 | Epoch Time: 1m 

KeyboardInterrupt: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [17]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


# ## Inference
# 
# We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

# In[ ]:







Test Loss: 0.875 | Test Acc: 60.78%


In [19]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    # ind = np.argmax(np.array(prediction))
    # if ind ==0:
    #   print('neutral')
    # elif ind == 1:
    #   print("positive")
    # else:
    #   print("negative")
    print(prediction)
predict_sentiment(model, tokenizer, "This film is terrible")


# In[ ]:


predict_sentiment(model, tokenizer, "This film is great")

tensor([[0.4050, 0.4728, 0.6445]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.6224, 0.8791, 0.2238]], device='cuda:0', grad_fn=<SigmoidBackward>)


In [0]:
while True:
  sent = input('->')
  if sent != '$':
    predict_sentiment(model, tokenizer, sent)
  else:
    break


tensor([[0.6313, 0.8916, 0.2122]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.6579, 0.4763, 0.4221]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.5785, 0.5827, 0.4829]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.6492, 0.5259, 0.4039]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.7715, 0.5227, 0.3111]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.7715, 0.5227, 0.3111]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.7893, 0.7136, 0.1800]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.5968, 0.5023, 0.4775]], device='cuda:0', grad_fn=<SigmoidBackward>)
